In [138]:
import argparse
import pysam
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
#ideally should work to functionalize everything 

In [139]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [140]:
'''Summary: A script to calculate the editing outcomes in BAM files at a given list of locations'''

'Summary: A script to calculate the editing outcomes in BAM files at a given list of locations'

In [141]:
strandedness = []
fasta = pysam.FastaFile("hg38.fa")

In [216]:
the_read = None
#the issue: crispresso doesn't quite work for WGS sequencing due to reads not being long enoughc

#fetch the reads that are between the regions of interest
#overall structure of the script (what it needs to do)
#1. traverse each read 3' to 5'
#2. if a deletion or insertion that overlaps with the PAM include it


#A BAM (Binary Alignment Map) file simply stores the aligned sequencing reads from a DNA sample, which are naturally oriented 5' to 3'
#The PAM is typically on the 3' end.


#something about my calculation of total reads must be wrong :( 

#find the locations of the deletions in a given BAM file for each read
def find_deletion_locs(inbam, chrom, start_loc, end_loc,strand):
    deletions = []
    insertions = []
    #table of all indels (-) for a feletion (+) for an insertion
    indels = []
    count = 0
    
    for read in inbam.fetch(chrom, start_loc, end_loc):  
        #if read is reverse the beginning of the region will be the 3' end
        #if the read is not in reverse the beginning of the region will be the 5' end i believe 
        deletion_lst = []
        insertion_lst = []
        indel_lst = []

        count += 1
        ref_start = read.reference_start  # Correct attribute for reference start position

        #if the start of the read is soft clipped
        if read.cigartuples[0][0] == 4:
            #make the location of the read start at the reference minus the length of the soft clipping as reference_start doesn't
            #otherwise include the soft clipped bases 
            ref_start = ref_start - read.cigartuples[0][1] 
        
        cigar = read.cigartuples
    
        #the bases traversed - where we are in the read 
        l_read = 0
        
        #the exact start position of the read - where we are in the reference 
        l = ref_start

        segment_index = 0

        #go through now the entire cigar array
        for op, length in cigar:
            #if we are overlapping with the guide region in the read or in between the guide region
                
            #if the read + the length of a deletion or insertion is passed the 3' end - count it as an overhang deletion

            '''below just checks if a contained or an overhang deletion'''
            #the deletion seems to be at the 5' end for whatever reason - a little confusing

            #not sure if there should be a an equal sign.... 
            
            #deletions that go past the guide but originate in the guide
            if strand == '-':
                #on the 3' end
                if length + l > start_loc and l < start_loc:
                    if op == 2:
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #on the 5' end
                elif l + length > end_loc and l < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                elif l > start_loc and l  < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                elif l + length > start_loc and l + length  < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])

            #deletion should be on the 3' end (at the end)
            if strand == '+' or strand == 'unknown':
                if length + l > end_loc and l < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                elif length + l > start_loc and l < start_loc:
                    if op == 2:
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                elif l > start_loc and l  < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                elif l + length > start_loc and l + length  < end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
            

            #insertion ends in the guide loc
            if l + length < end_loc and l + length > start_loc :
                if op == 1:
                    insertion_lst.append(["insertions", length, (l, l+length)])
                    indel_lst.append(["indels", length])
                    
            #insertion begins in the guide loc
            if l  < end_loc and l > start_loc :
                if op == 1:
                    insertion_lst.append(["insertions", length, (l, l+length)])
                    indel_lst.append(["indels", length])
                
            
            '''below traverses the region of the read we are considering'''
            if op == 0:  # NOT SURE WHAT THE MEANING OF THIS IS
                l_read += length
                l += length
                
            elif op == 1:  # Insertion (this is correct)
                l_read += length
                
            elif op == 2:  # Deletion (this is correct)
                l += length 
                
            elif op == 3:  # Skipped region
                l += length #moving ahead in the reference
                
            elif op == 4:  # Soft clipping
                l_read += length
                l += length #
                
            elif op == 5:  # Hard clipping NOT SURE 
                pass
            elif op == 6:  # Padding NOT SURE 
                pass
            elif op == 7:  # Segment equal (this is correct)
                l_read += length
                l += length
                
            elif op == 8:  # seems to be sequence mismatcj - they both advance then NOT SURE
                l_read += length
                l += length
                
            else:
                raise ValueError(f"Unknown CIGAR operation code {op}")

            #if - once adding the oppcode have suprased the end of the read - break
            if l > end_loc:
                if len(deletion_lst) > 0 or len(insertion_lst) > 0:
                    deletions.append(deletion_lst)
                    insertions.append(insertion_lst)
                break
                
            segment_index += 1
            

    #returns a list of all the deletions and all the insertions and a total indel lst
    return deletions, insertions, indels
            
    # Close the BAM file
    inbam.close()

#construct the DF
def construct_df(locations, bams, del_lst, ins_lst, indel_lst):
    locations = locations.copy()    
    for region in bams:
        inbam = bams[region]
        chrom = vegfa_df[vegfa_df['REGION'] == region]['chrom'].tolist()[0]
        start = vegfa_df[vegfa_df['REGION'] == region]['start'].tolist()[0]
        end = vegfa_df[vegfa_df['REGION'] == region]['end'].tolist()[0]

        deletions = del_lst[region]
        indels = indel_lst[region]
        
        deletion_lengths = [
            [deletion[1] for deletion in read if 'deletions' in deletion[0]]
            for read in deletions
        ]

        deletion_coords = [
            [deletion[2] for deletion in read if 'deletions' in deletion[0]]
            for read in deletions
        ]

        indel_lengths = [
            [indel[1] for indel in read if 'indel' in indel[0]]
            for read in indels
        ]
        
        num_deletions = len(deletion_lengths)

        deletion_extending_past_region_lengths = [deletion[1] for read in deletions for deletion in read if 'extending' in deletion[0]]
        deletion_extending_past_region_coords = [deletion[2] for read in deletions for deletion in read if 'extending' in deletion[0]]
        num_deletions_extending_past_region = len(deletion_extending_past_region_coords)
        
        insertions = ins_lst[region]
        insertion_lengths = [insertion[1] for read in insertions for insertion in read if 'insertions' == insertion[0]]
        insertion_coords = [insertion[2] for read in insertions for insertion in read if 'insertions' == insertion[0]]
        num_insertions = len(insertion_coords)  
        num_indels = len(indel_lengths)

        #adding some columns to the dataframes
        total_reads = sum(1 for read in inbam.fetch(chrom, start, end))
        locations.loc[locations['REGION'] == region, 'total_reads'] = total_reads
        locations.loc[locations['REGION'] == region, 'forward_reads'] = sum(1 for read in inbam.fetch(chrom, start, end) if not read.is_reverse)
        locations.loc[locations['REGION'] == region, 'reverse_reads'] = sum(1 for read in inbam.fetch(chrom, start, end) if read.is_reverse)

        #collect info about all deletions
        locations.loc[locations['REGION'] == region, 'deletion lengths'] = str(deletion_lengths)
        locations.loc[locations['REGION'] == region, 'deletion coords'] = str(deletion_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with deletion(s)'] = str(num_deletions)

        #collect info about all insertions
        locations.loc[locations['REGION'] == region, 'insertion lengths'] = str(insertion_lengths)
        locations.loc[locations['REGION'] == region, 'insertion coords'] = str(insertion_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with insertion(s)'] = str(num_insertions)
        locations.loc[locations['REGION'] == region, 'number of reads with indels'] = str(num_indels)     


        #collect info about all deletions that go past the guide site
        locations.loc[locations['REGION'] == region, 'deletion extending past region lengths'] = str(deletion_extending_past_region_lengths)
        locations.loc[locations['REGION'] == region, 'deletion extending past region coords'] = str(deletion_extending_past_region_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with deletion extending past region'] = str(num_deletions_extending_past_region)     

        if total_reads != 0:
            locations.loc[locations['REGION'] == region, '% reads with an deletion'] = int(num_deletions) / int(total_reads)
            locations.loc[locations['REGION'] == region, '% reads with an insertion'] = int(num_insertions) / int(total_reads)
            locations.loc[locations['REGION'] == region, '% reads with an indel'] = int(num_indels) / int(total_reads)
        else:
            locations.loc[locations['REGION'] == region, '% reads with an insertion'] = None
            locations.loc[locations['REGION'] == region, '% reads with an deletion'] = None
            locations.loc[locations['REGION'] == region, '% reads with an indel'] = None
            
    return locations

In [217]:
#for each noted location - could see which one has more homology with

In [218]:
start_loc=43770818
end_loc=43770841

#need to populate a list perhaps with all the start and end locations
vegfa = 'vegfa_lifted_2.tsv'
vegfa_df = pd.read_csv(vegfa, sep='\t',  names=['chrom','start','end','name','na','na2','na3']).reset_index()
# Rename all columns
vegfa_df.columns = ['chrom', 'start', 'end', 'name', 'off_target', 'na', 'na2', 'REGION']

# Print the DataFrame to verify
print(vegfa_df)

#read in the csv file
vegfa_df

import pysam
from pathlib import Path

# Define the paths to the BAM directories
HG_1_BAMs = Path('CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide') / 'ANALYZED_REGIONS'
HG_2_BAMs = Path('CRISPRessoWGS_on_H9_2_combined_crispresso_wo_guide') / 'ANALYZED_REGIONS'
HG_WT_BAMs = Path('CRISPRessoWGS_on_H9_WT_combined_crispresso_wo_guide') / 'ANALYZED_REGIONS'

# Function to read BAM files from a directory and store in a dictionary
def read_bam_files_to_dict(directory):
    #sort the directory by name
    bam_files_dict = {}
    
    sorted_bam_files = sorted(directory.glob('*.bam'), key=lambda f: f.name)
    
    for bam_file in sorted_bam_files:  # Filter for .bam files
        bam_files_dict[bam_file.name] = pysam.AlignmentFile(bam_file, "rb")  # Store filename as key, BAM object as value
    return bam_files_dict

# Read BAM files from each directory
bam_files_HG_1 = read_bam_files_to_dict(HG_1_BAMs)
bam_files_HG_2 = read_bam_files_to_dict(HG_2_BAMs)
bam_files_HG_WT = read_bam_files_to_dict(HG_WT_BAMs)

# Combine both dictionaries
all_bam_files = {**bam_files_HG_1, **bam_files_HG_2}

# Print a summary
print(f"Number of BAM files in HG_1: {len(bam_files_HG_1)}")
print(f"Number of BAM files in HG_2: {len(bam_files_HG_2)}")
print(f"Number of BAM files in HG_2: {len(bam_files_HG_WT)}")
print(f"Total number of BAM files: {len(all_bam_files)}")

print(vegfa.index)

#Add a column to the vegfa table where each region cooresponds to a REGION file - so can be easily paired up with the region BAM files
vegfa_df['REGION'] = ['REGION_' + str(i) + '.bam' for i in np.arange(0, vegfa_df.shape[0])]
vegfa_df

     chrom      start        end                             name  \
0     chr5    6714988    6715011    VEGFA_site2_chr5_6715118_1925   
1     chr2  241275174  241275197  VEGFA_site2_chr2_242214606_1549   
2     chr9  100837360  100837383  VEGFA_site2_chr9_103599649_1178   
3    chr16   56929510   56929533  VEGFA_site2_chr16_56963429_1107   
4     chr4   38535989   38536012   VEGFA_site2_chr4_38537627_1054   
5    chr11   72237743   72237766   VEGFA_site2_chr11_71948805_919   
6    chr17    4455450    4455473    VEGFA_site2_chr17_4358752_803   
7    chr17   41888497   41888520   VEGFA_site2_chr17_40044757_791   
8     chr9   27338859   27338882    VEGFA_site2_chr9_27338874_767   
9     chr8  143740774  143740797   VEGFA_site2_chr8_144822961_728   
10   chr10  114534490  114534513  VEGFA_site2_chr10_116294256_704   
11   chr22   50446356   50446379   VEGFA_site2_chr22_50884803_704   
12   chr11   76784738   76784761   VEGFA_site2_chr11_76495788_627   
13    chr1  151059392  151059415  

[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_1.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_10.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_102.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_103.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_104.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispresso_wo_guide/ANALYZED_REGIONS/REGION_106.bam.bai
[W::hts_idx_load3] The index file is older than the data file: CRISPRessoWGS_on_H9_1_combined_crispress

Number of BAM files in HG_1: 152
Number of BAM files in HG_2: 152
Number of BAM files in HG_2: 152
Total number of BAM files: 152
<built-in method index of str object at 0x143047f30>


,chrom,start,end,name,off_target,na,na2,REGION
0,chr5,6714988,6715011,VEGFA_site2_chr5_6715118_1925,CTACCCCTCCACCCCGCCTCCGG,NaN,NaN,REGION_0.bam
1,chr2,241275174,241275197,VEGFA_site2_chr2_242214606_1549,ATTCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_1.bam
2,chr9,100837360,100837383,VEGFA_site2_chr9_103599649_1178,ACACCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_2.bam
3,chr16,56929510,56929533,VEGFA_site2_chr16_56963429_1107,TGCCCCCCCCACCCCACCTCTGG,NaN,NaN,REGION_3.bam
4,chr4,38535989,38536012,VEGFA_site2_chr4_38537627_1054,CTCCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_4.bam
5,chr11,72237743,72237766,VEGFA_site2_chr11_71948805_919,GCTTCCCTCCACCCCGCATCCGG,NaN,NaN,REGION_5.bam
6,chr17,4455450,4455473,VEGFA_site2_chr17_4358752_803,TACCCCCCACACCCCGCCTCTGG,NaN,NaN,REGION_6.bam
7,chr17,41888497,41888520,VEGFA_site2_chr17_40044757_791,TGCCCCTCCCACCCCGCCTCTGG,NaN,NaN,REGION_7.bam
8,chr9,27338859,27338882,VEGFA_site2_chr9_27338874_767,GACCCCTCCCACCCCGACTCCGG,NaN,NaN,REGION_8.bam
9,chr8,143740774,143740797,VEGFA_site2_chr8_144822961_728,GTACCCCACCACCCCGCCCCAGG,NaN,NaN,REGION_9.bam


In [219]:
#figure out the strandedness of each off target region and add that to the table

#not quite sure why some of the off-targets dpn't match exactly
strandedness = []

for index, row in vegfa_df.iterrows():
    off_target_seq = row['off_target'].upper()
    ref_sequence = fasta.fetch(row['chrom'], row['start'], row['end']).upper()

    if ref_sequence == off_target_seq:
        strand = '+'
    elif ref_sequence[::-1].translate(str.maketrans("ACGT", "TGCA")) == off_target_seq:
        strand = '-'
    else:
        print(ref_sequence)
        print(off_target_seq)
        print('\n')
        strand = "unknown"
    strandedness.append(strand)
strandedness


vegfa_df['strandedness'] = strandedness

vegfa_df

CTCCCCACCCACCCCGCCTCAGG
CTCCCCCCCCACCCCGCCTCAGG


TCCCCCCCACCCCCGACCTCAGG
TCCCCCCCACCCCCCACCTCAGG


CCCGGGGCGGAATGGGGAGGGCG
CGCCCTCCCCACTCCGCCTCGGG


CCGGGGGCGGGGAGGGGGGGCAG
TGCCCCCTCT-CCCCGCCCCCGG


GTCCCCCTCCTCCCTGTCTCAGG
GTCCCCCTCCTCCCCGTCTCAGG


CCAGAGATGGGGTGGGGGTGGGG
CCCCACCCCCACCCCACCTCTGG




,chrom,start,end,name,off_target,na,na2,REGION,strandedness
0,chr5,6714988,6715011,VEGFA_site2_chr5_6715118_1925,CTACCCCTCCACCCCGCCTCCGG,NaN,NaN,REGION_0.bam,+
1,chr2,241275174,241275197,VEGFA_site2_chr2_242214606_1549,ATTCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_1.bam,+
2,chr9,100837360,100837383,VEGFA_site2_chr9_103599649_1178,ACACCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_2.bam,-
3,chr16,56929510,56929533,VEGFA_site2_chr16_56963429_1107,TGCCCCCCCCACCCCACCTCTGG,NaN,NaN,REGION_3.bam,-
4,chr4,38535989,38536012,VEGFA_site2_chr4_38537627_1054,CTCCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_4.bam,unknown
5,chr11,72237743,72237766,VEGFA_site2_chr11_71948805_919,GCTTCCCTCCACCCCGCATCCGG,NaN,NaN,REGION_5.bam,+
6,chr17,4455450,4455473,VEGFA_site2_chr17_4358752_803,TACCCCCCACACCCCGCCTCTGG,NaN,NaN,REGION_6.bam,-
7,chr17,41888497,41888520,VEGFA_site2_chr17_40044757_791,TGCCCCTCCCACCCCGCCTCTGG,NaN,NaN,REGION_7.bam,-
8,chr9,27338859,27338882,VEGFA_site2_chr9_27338874_767,GACCCCTCCCACCCCGACTCCGG,NaN,NaN,REGION_8.bam,+
9,chr8,143740774,143740797,VEGFA_site2_chr8_144822961_728,GTACCCCACCACCCCGCCCCAGG,NaN,NaN,REGION_9.bam,+


In [220]:
del_dict_hg1 = {}
ins_dict_hg1 = {}
indel_dict_hg1 = {}

for bam in bam_files_HG_1:
    #find the deletions
    chrom = vegfa_df[vegfa_df['REGION'] == bam]['chrom'].tolist()[0]
    start = vegfa_df[vegfa_df['REGION'] == bam]['start'].tolist()[0]
    end = vegfa_df[vegfa_df['REGION'] == bam]['end'].tolist()[0]
    strand = vegfa_df[vegfa_df['REGION'] == bam]['strandedness'].tolist()[0]
    
    del_dict_hg1[bam], ins_dict_hg1[bam], indel_dict_hg1[bam] = find_deletion_locs(bam_files_HG_1[bam], chrom, start, end, strand)

del_dict_hg2 = {}
ins_dict_hg2 = {}
indel_dict_hg2 = {}
for bam in bam_files_HG_2:
    #find the deletions
    chrom = vegfa_df[vegfa_df['REGION'] == bam]['chrom'].tolist()[0]
    start = vegfa_df[vegfa_df['REGION'] == bam]['start'].tolist()[0]
    end = vegfa_df[vegfa_df['REGION'] == bam]['end'].tolist()[0]
    strand = vegfa_df[vegfa_df['REGION'] == bam]['strandedness'].tolist()[0]
    del_dict_hg2[bam], ins_dict_hg2[bam], indel_dict_hg2[bam] = find_deletion_locs(bam_files_HG_2[bam], chrom, start, end, strand)

del_dict_hgwt = {}
ins_dict_hgwt = {}
indel_dict_hgwt = {}
for bam in bam_files_HG_WT:
    #find the deletions
    chrom = vegfa_df[vegfa_df['REGION'] == bam]['chrom'].tolist()[0]
    start = vegfa_df[vegfa_df['REGION'] == bam]['start'].tolist()[0]
    end = vegfa_df[vegfa_df['REGION'] == bam]['end'].tolist()[0]
    strand = vegfa_df[vegfa_df['REGION'] == bam]['strandedness'].tolist()[0]
    del_dict_hgwt[bam], ins_dict_hgwt[bam], indel_dict_hgwt[bam] = find_deletion_locs(bam_files_HG_WT[bam], chrom, start, end, strand)

#lots of the regions have small "indels"
#but few seem to have full on excisions that go passed the guide 
del_dict_hg1['REGION_15.bam']

[[['deletions extending passed region', 56, (43770767, 43770823)]],
 [['deletions', 1, (43770834, 43770835)]],
 [['deletions extending passed region', 49, (43770777, 43770826)]],
 [['deletions extending passed region', 49, (43770777, 43770826)]],
 [['deletions extending passed region', 10, (43770814, 43770824)]],
 [['deletions extending passed region', 43, (43770781, 43770824)]],
 [['deletions extending passed region', 43, (43770781, 43770824)]],
 [['deletions', 3, (43770823, 43770826)]],
 [['deletions extending passed region', 25, (43770801, 43770826)]],
 [['deletions extending passed region', 43, (43770781, 43770824)]],
 [['deletions extending passed region', 18, (43770806, 43770824)]],
 [['deletions extending passed region', 30, (43770794, 43770824)]],
 [['deletions', 1, (43770823, 43770824)]],
 [['deletions extending passed region', 35, (43770811, 43770846)]],
 [['deletions extending passed region', 30, (43770794, 43770824)]],
 [['deletions extending passed region', 30, (43770794, 

In [221]:
#construct the two tables 
hg2tbl = construct_df(vegfa_df,bam_files_HG_2, del_dict_hg2, ins_dict_hg2, indel_dict_hg2)
hg1tbl = construct_df(vegfa_df,bam_files_HG_1, del_dict_hg1, ins_dict_hg1, indel_dict_hg1)
hgwttbl = construct_df(vegfa_df,bam_files_HG_WT, del_dict_hgwt, ins_dict_hgwt, indel_dict_hgwt)

# # Save hg_2_tbl_dec5 to a CSV file
hg2tbl.to_csv("hg_2_tbl_dec5.csv", index=False)
# # Save hg_1_tbl_dec5 to a CSV file
hg1tbl.to_csv("hg_1_tbl_dec5.csv", index=False)
# # Save hg_WT_tbl_dec5 to a CSV file
hgwttbl.to_csv("hg_1_tbl_dec5.csv", index=False)


In [222]:
hgwttbl

,chrom,start,end,name,off_target,na,na2,REGION,strandedness,total_reads,forward_reads,reverse_reads,deletion lengths,deletion coords,number of reads with deletion(s),insertion lengths,insertion coords,number of reads with insertion(s),number of reads with indels,deletion extending past region lengths,deletion extending past region coords,number of reads with deletion extending past region,% reads with an deletion,% reads with an insertion,% reads with an indel
0,chr5,6714988,6715011,VEGFA_site2_chr5_6715118_1925,CTACCCCTCCACCCCGCCTCCGG,NaN,NaN,REGION_0.bam,+,37.0,19.0,18.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
1,chr2,241275174,241275197,VEGFA_site2_chr2_242214606_1549,ATTCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_1.bam,+,32.0,14.0,18.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
2,chr9,100837360,100837383,VEGFA_site2_chr9_103599649_1178,ACACCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_2.bam,-,43.0,14.0,29.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
3,chr16,56929510,56929533,VEGFA_site2_chr16_56963429_1107,TGCCCCCCCCACCCCACCTCTGG,NaN,NaN,REGION_3.bam,-,64.0,25.0,39.0,[[]],[[]],1,"[1, 1]","[(56929523, 56929524), (56929523, 56929524)]",2,0,[],[],0,0.015625,0.031250,0.0
4,chr4,38535989,38536012,VEGFA_site2_chr4_38537627_1054,CTCCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_4.bam,unknown,13.0,0.0,13.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
5,chr11,72237743,72237766,VEGFA_site2_chr11_71948805_919,GCTTCCCTCCACCCCGCATCCGG,NaN,NaN,REGION_5.bam,+,36.0,15.0,21.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
6,chr17,4455450,4455473,VEGFA_site2_chr17_4358752_803,TACCCCCCACACCCCGCCTCTGG,NaN,NaN,REGION_6.bam,-,26.0,14.0,12.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
7,chr17,41888497,41888520,VEGFA_site2_chr17_40044757_791,TGCCCCTCCCACCCCGCCTCTGG,NaN,NaN,REGION_7.bam,-,24.0,11.0,13.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
8,chr9,27338859,27338882,VEGFA_site2_chr9_27338874_767,GACCCCTCCCACCCCGACTCCGG,NaN,NaN,REGION_8.bam,+,40.0,21.0,19.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
9,chr8,143740774,143740797,VEGFA_site2_chr8_144822961_728,GTACCCCACCACCCCGCCCCAGG,NaN,NaN,REGION_9.bam,+,22.0,13.0,9.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0


In [223]:
# display(hg1tbl[hg1tbl['REGION'] == 'REGION_15.bam'])

#lets see which one has the maximum deletion rate
print(hg1tbl['% reads with an deletion'])

max_row_del = hg1tbl.loc[hg1tbl['% reads with an deletion'].idxmax()]
max_row_ins = hg1tbl.loc[hg1tbl['% reads with an insertion'].idxmax()]

#ratio of deletion rate across edited and non-edited sample complete with an FDR calculation

0      0.043605
1      0.012012
2      0.015267
3      0.020690
4      0.000000
5      0.002762
6      0.007143
7      0.000000
8      0.000000
9      0.000000
10     0.007463
11     0.436261
12     0.488462
13     0.002967
14     0.002801
15     0.058309
16     0.033854
17     0.018727
18     0.000000
19     0.077206
20     0.795518
21     0.000000
22     0.012920
23     0.071685
24     0.000000
25     0.421296
26     0.000000
27     0.008772
28     0.043919
29     0.008547
30     0.015152
31     0.014235
32     0.002849
33     0.000000
34     0.002681
35     0.000000
36     0.007092
37     0.000000
38     0.037415
39     0.000000
40     0.013774
41     0.000000
42     0.016000
43     0.000000
44     0.000000
45     0.000000
46     0.004785
47     0.044025
48     0.000000
49     0.000000
50     0.000000
51     0.340909
52     0.013115
53     0.011142
54     0.000000
55     0.002703
56     0.014085
57     0.037500
58     0.002950
59     0.003289
60     0.000000
61     0.000000
62     0

In [224]:
#save all the csv files - and verify results

In [225]:
#right now only keeps track of deletions - can make it keep track of insertions too

#iterate through each region
for region in del_dict_hg1:
    #iterate through each read
    print(region)
    for read in del_dict_hg1[region]:
        #iterate through each deletion type in the read
        for dels in read:      
            if not dels[0] == 'contained deletion':
                print('del', dels)
    print('\n')

REGION_0.bam
del ['deletions', 6, (6714996, 6715002)]
del ['deletions', 5, (6714999, 6715004)]
del ['deletions', 18, (6714992, 6715010)]
del ['deletions', 18, (6714992, 6715010)]
del ['deletions', 17, (6714988, 6715005)]
del ['deletions', 17, (6714988, 6715005)]
del ['deletions', 17, (6714988, 6715005)]
del ['deletions', 17, (6714988, 6715005)]
del ['deletions', 8, (6715003, 6715011)]
del ['deletions', 4, (6715004, 6715008)]


REGION_1.bam
del ['deletions', 1, (241275190, 241275191)]
del ['deletions', 1, (241275190, 241275191)]
del ['deletions extending passed region', 27, (241275183, 241275210)]


REGION_10.bam


REGION_100.bam


REGION_101.bam
del ['deletions', 2, (43369587, 43369589)]
del ['deletions', 2, (43369587, 43369589)]
del ['deletions', 2, (43369587, 43369589)]
del ['deletions', 2, (43369587, 43369589)]
del ['deletions', 2, (43369587, 43369589)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
de

In [226]:
#iterate through each region
for region in del_dict_hg2:
    #iterate through each read
    print(region)
    for read in del_dict_hg2[region]:
        #iterate through each deletion type in the read
        for dels in read:      
            if not dels[0] == 'contained deletion':
                print('del', dels)
    print('\n')

REGION_0.bam
del ['deletions', 6, (6714999, 6715005)]
del ['deletions', 6, (6715001, 6715007)]
del ['deletions', 8, (6715003, 6715011)]
del ['deletions', 8, (6715003, 6715011)]
del ['deletions', 6, (6715001, 6715007)]
del ['deletions', 6, (6715001, 6715007)]
del ['deletions', 8, (6715003, 6715011)]
del ['deletions', 8, (6715003, 6715011)]
del ['deletions', 8, (6715003, 6715011)]


REGION_1.bam
del ['deletions', 12, (241275181, 241275193)]


REGION_10.bam
del ['deletions', 1, (114534508, 114534509)]


REGION_100.bam
del ['deletions', 1, (41575710, 41575711)]


REGION_101.bam
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369587, 43369588)]
del ['deletions', 1, (43369586, 43369587)]
del ['deletions', 1, (43369587, 

In [227]:
hg1tbl

,chrom,start,end,name,off_target,na,na2,REGION,strandedness,total_reads,forward_reads,reverse_reads,deletion lengths,deletion coords,number of reads with deletion(s),insertion lengths,insertion coords,number of reads with insertion(s),number of reads with indels,deletion extending past region lengths,deletion extending past region coords,number of reads with deletion extending past region,% reads with an deletion,% reads with an insertion,% reads with an indel
0,chr5,6714988,6715011,VEGFA_site2_chr5_6715118_1925,CTACCCCTCCACCCCGCCTCCGG,NaN,NaN,REGION_0.bam,+,344.0,159.0,185.0,"[[6], [5], [18], [18], [], [], [], [], [], [17...","[[(6714996, 6715002)], [(6714999, 6715004)], [...",15,"[6, 6, 17, 12, 12, 2, 2, 10, 4, 4, 10]","[(6715004, 6715010), (6715004, 6715010), (6715...",11,0,[],[],0,0.043605,0.031977,0.0
1,chr2,241275174,241275197,VEGFA_site2_chr2_242214606_1549,ATTCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_1.bam,+,333.0,193.0,140.0,"[[1], [1], [], [27]]","[[(241275190, 241275191)], [(241275190, 241275...",4,"[1, 1]","[(241275177, 241275178), (241275177, 241275178)]",2,0,[27],"[(241275183, 241275210)]",1,0.012012,0.006006,0.0
2,chr9,100837360,100837383,VEGFA_site2_chr9_103599649_1178,ACACCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_2.bam,-,393.0,182.0,211.0,"[[3], [3], [3], [3], [12], [12]]","[[(100837365, 100837368)], [(100837363, 100837...",6,[],[],0,0,[],[],0,0.015267,0.000000,0.0
3,chr16,56929510,56929533,VEGFA_site2_chr16_56963429_1107,TGCCCCCCCCACCCCACCTCTGG,NaN,NaN,REGION_3.bam,-,290.0,132.0,158.0,"[[1], [1], [1], [3], [], []]","[[(56929523, 56929524)], [(56929523, 56929524)...",6,"[1, 1, 1, 1]","[(56929523, 56929524), (56929523, 56929524), (...",4,0,[],[],0,0.020690,0.013793,0.0
4,chr4,38535989,38536012,VEGFA_site2_chr4_38537627_1054,CTCCCCCCCCACCCCGCCTCAGG,NaN,NaN,REGION_4.bam,unknown,180.0,22.0,158.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
5,chr11,72237743,72237766,VEGFA_site2_chr11_71948805_919,GCTTCCCTCCACCCCGCATCCGG,NaN,NaN,REGION_5.bam,+,362.0,182.0,180.0,[[11]],"[[(72237754, 72237765)]]",1,[],[],0,0,[],[],0,0.002762,0.000000,0.0
6,chr17,4455450,4455473,VEGFA_site2_chr17_4358752_803,TACCCCCCACACCCCGCCTCTGG,NaN,NaN,REGION_6.bam,-,420.0,171.0,249.0,"[[8], [1], [5]]","[[(4455448, 4455456)], [(4455465, 4455466)], [...",3,[],[],0,0,[8],"[(4455448, 4455456)]",1,0.007143,0.000000,0.0
7,chr17,41888497,41888520,VEGFA_site2_chr17_40044757_791,TGCCCCTCCCACCCCGCCTCTGG,NaN,NaN,REGION_7.bam,-,496.0,252.0,244.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
8,chr9,27338859,27338882,VEGFA_site2_chr9_27338874_767,GACCCCTCCCACCCCGACTCCGG,NaN,NaN,REGION_8.bam,+,406.0,179.0,227.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0
9,chr8,143740774,143740797,VEGFA_site2_chr8_144822961_728,GTACCCCACCACCCCGCCCCAGG,NaN,NaN,REGION_9.bam,+,198.0,111.0,87.0,[],[],0,[],[],0,0,[],[],0,0.000000,0.000000,0.0


In [228]:
#make a table where we now compare all the editing outcomes - ratio of deletion to deletions, insertions to insertions

#takes in a WT and a edited sample and compares the deletion and insertion rate - as well as calculates a p-value of sorts
#signficant difference in the rates (or the distributions?)
#depth of the WT sample as of now is not great so

def construct_comp_tbl(wt, edited):
    comp_df = pd.DataFrame([])
    # Assuming wt and edited are pandas DataFrames with the same structure
    comp_df['REGION'] = wt['REGION'].tolist()
    print(wt['% reads with an deletion'])
    print(edited['% reads with an deletion'])
    comp_df['% edited/ % wt deletion rate'] = edited['% reads with an deletion'] / wt['% reads with an deletion']
    comp_df['% edited/ % wt insertion rate'] = edited['% reads with an insertion'] / wt['% reads with an insertion']
    return comp_df

In [229]:
#the data is "bernoulli" data therefore 
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
import scipy.stats as stats


def construct_comp_tbl(wt, edited):
    comp_df = pd.DataFrame([])
    
    # Ensure 'REGION' column is included
    comp_df['REGION'] = wt['REGION'].tolist()
    
    # Handle division carefully, replacing zeros with NaN to avoid infinite values
    comp_df['% edited / % wt deletion rate'] = np.where(
        wt['% reads with an deletion'] != 0,
        edited['% reads with an deletion'] / wt['% reads with an deletion'],
        np.nan
    )
    comp_df['% edited / % wt insertion rate'] = np.where(
        wt['% reads with an insertion'] != 0,
        edited['% reads with an insertion'] / wt['% reads with an insertion'],
        np.nan
    )

    #construct a contigency table
    for index, row in wt.iterrows():
        cont = pd.DataFrame([])
        cont_indel = pd.DataFrame([])
        
        region = row['REGION']
        cont['wt'] = [int(row['number of reads with deletion(s)']),
        int(row['total_reads']) - int(row['number of reads with deletion(s)'])]

        row2 = edited.iloc[index]
        cont['edited'] = [int(row2['number of reads with deletion(s)']),
        int(row2['total_reads']) - int(row2['number of reads with deletion(s)'])]
        
        cont['type'] = ['deletion', 'no deletions']
        cont.set_index('type', inplace=True)
        display(cont)
        #not quite significant -- perhaps because don't have enough depth
        #now that we have constructed the contigency tables - do some stats !
        odds_ratio, p_value = stats.fisher_exact(cont)

        
        #create the regions table
        comp_df.loc[comp_df['REGION'] == region, 'deletion odds_ratio'] = odds_ratio
        comp_df.loc[comp_df['REGION'] == region, 'deletion p-value'] = p_value
        
        comp_df.loc[comp_df['REGION'] == region, 'indel odds_ratio'] = odds_ratio
        comp_df.loc[comp_df['REGION'] == region, 'indel p-value'] = p_value
    
    return comp_df
    
    #calculate the chi squared test 

#probably non of the stats are really significant
display(construct_comp_tbl(hgwttbl, hg1tbl))

,wt,edited
type,,
deletion,0,15
no deletions,37,329


,wt,edited
type,,
deletion,0,4
no deletions,32,329


,wt,edited
type,,
deletion,0,6
no deletions,43,387


,wt,edited
type,,
deletion,1,6
no deletions,63,284


,wt,edited
type,,
deletion,0,0
no deletions,13,180


...

,wt,edited
type,,
deletion,0,0
no deletions,33,288


,wt,edited
type,,
deletion,0,0
no deletions,27,294


,wt,edited
type,,
deletion,0,0
no deletions,27,383


,wt,edited
type,,
deletion,1,1
no deletions,27,351


,REGION,% edited / % wt deletion rate,% edited / % wt insertion rate,deletion odds_ratio,deletion p-value,indel odds_ratio,indel p-value
0,REGION_0.bam,NaN,NaN,0.000000,0.379283,0.000000,0.379283
1,REGION_1.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
2,REGION_2.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
3,REGION_3.bam,1.324138,0.441379,0.751323,1.000000,0.751323,1.000000
4,REGION_4.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
5,REGION_5.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
6,REGION_6.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
7,REGION_7.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
8,REGION_8.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
9,REGION_9.bam,NaN,NaN,NaN,1.000000,NaN,1.000000


In [230]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#can also do something with the distribution of the 
construct_comp_tbl(hgwttbl, hg1tbl)

,wt,edited
type,,
deletion,0,15
no deletions,37,329


,wt,edited
type,,
deletion,0,4
no deletions,32,329


,wt,edited
type,,
deletion,0,6
no deletions,43,387


,wt,edited
type,,
deletion,1,6
no deletions,63,284


,wt,edited
type,,
deletion,0,0
no deletions,13,180


...

,wt,edited
type,,
deletion,0,0
no deletions,33,288


,wt,edited
type,,
deletion,0,0
no deletions,27,294


,wt,edited
type,,
deletion,0,0
no deletions,27,383


,wt,edited
type,,
deletion,1,1
no deletions,27,351


,REGION,% edited / % wt deletion rate,% edited / % wt insertion rate,deletion odds_ratio,deletion p-value,indel odds_ratio,indel p-value
0,REGION_0.bam,NaN,NaN,0.000000,0.379283,0.000000,0.379283
1,REGION_1.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
2,REGION_2.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
3,REGION_3.bam,1.324138,0.441379,0.751323,1.000000,0.751323,1.000000
4,REGION_4.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
5,REGION_5.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
6,REGION_6.bam,NaN,NaN,0.000000,1.000000,0.000000,1.000000
7,REGION_7.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
8,REGION_8.bam,NaN,NaN,NaN,1.000000,NaN,1.000000
9,REGION_9.bam,NaN,NaN,NaN,1.000000,NaN,1.000000


$$ \chi^2 = \sum \frac{(O_i - E_i)^2}{E_i} $$
$$ E_i = \frac{( \text{Row Total} \times \text{Column Total} )}{ \text{Grand Total} } $$
(the formula for a chi squared test - where O_i is the observed frequency and E_i is the expected frequency for a category - summation over all categories)

In [231]:
#there are 12 dleetion cords
#seems to be 20 deletions.... 
print(hgwttbl[hgwttbl['REGION'] ==  "REGION_12.bam"]['deletion coords'].tolist()[0])


[[(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)], [(76784759, 76784760)]]


ValueError: invalid literal for int() with base 10: '[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]'

In [ ]:
#light verification of the results..... bleh